# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

In [ ]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-08-29 10:37:22--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   168MB/s    in 0.3s    

2024-08-29 10:37:23 (168 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
replace imdb-reviews-pt-br.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Instalação manual das dependências para uso do pyspark no Google Colab

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a2c243f6f8b747440d8ca8c90f359f01a453bcb532864efdb4dfae1b2c59652a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


## Importar, instanciar e criar a SparkSession

In [ ]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

In [ ]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

In [ ]:
def map1(x):
  if x['sentiment'] == 'neg':
    return ('neg', int(x['id']))
  else:
    return ('other', 0)

## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

In [ ]:
def reduceByKey1(x,y):
  return x + y

## Aplicação do map/reduce e visualização do resultado

In [ ]:
ru = 0000000
rdd = imdb_df.rdd

# Aplicando a função de map
mapped_rdd = rdd.map(map1)

# Filtrando apenas os sentimentos negativos
negative_rdd = mapped_rdd.filter(lambda x: x[0] == 'neg')

# Aplicando a função de reduce
result = negative_rdd.map(lambda x: x[1]).reduce(reduceByKey1)

# Visualizando o resultado
result_df = spark.createDataFrame([(result,ru)], ["Soma dos IDs Negativos", "RU"])
result_df.show()

In [ ]:
##VERIFICACAO DO RESULTADO EM PYTHON
#import pandas as pd

def soma_ids_negativos(csv_file):
    # Carregar o arquivo CSV
    df = pd.read_csv('imdb-reviews-pt-br.csv')

    # Filtrar os filmes com avaliação negativa
    negativos = df[df['sentiment'] == 'neg']

    # Somar os valores da coluna 'id'
    soma_ids = negativos['id'].sum()

    return soma_ids

# Exemplo de uso
csv_file = 'imdb-reviews-pt-br.csv'
resultado = soma_ids_negativos(csv_file)
print(f"A soma dos IDs dos filmes classificados como negativos é: {resultado}")


# Questão 2:

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e
uma tupla com a soma das palavras de cada texto como valor.

In [ ]:
def map2(row):
  if row['sentiment'] == 'neg':
    words_count_pt = len(re.findall(r'\w+', row['text_pt']))
    words_count_en = len(re.findall(r'\w+', row['text_en']))
    return ('neg', (words_count_en, words_count_pt))
  else:
    return ('other', (0,0))

## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment".

In [ ]:
def reduceByKey2(x,y):
  # x e y são tuplas no formato ('sentiment', (words_count_pt, words_count_en))
  #sentiment = x[0]
  total_words_pt = x[0] + y[0]
  total_words_en = x[1] + y[1]
  return (total_words_pt, total_words_en)

## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [ ]:
# Coloque aqui suas linhas de código final
# "id","text_en","text_pt","sentiment" colunas do csv

import re # Import the regular expression module
rdd2 = imdb_df.rdd
ru = 0000000

# Aplicando a função de map
mapped_rdd2 = rdd2.map(lambda row: map2(row.asDict()))

# Filtrando apenas os sentimentos negativos
negative_rdd2 = mapped_rdd2.filter(lambda x: x[0] == 'neg')

# Aplicando a função de reduce
reduced_rdd2 = negative_rdd2.map(lambda x: x[1]).reduce(reduceByKey2)

# Coletando e visualizando o resultado
total_words_pt, total_words_en = reduced_rdd2

# Calculando a diferença entre palavras em português e inglês
difference = total_words_pt - total_words_en

# Visualizando o resultado
result_df = spark.createDataFrame([(difference,ru)], ["Diferença de palavras (português - inglês):", "RU"])
result_df.show()

In [ ]:
##VERIFICACAO EM PYTHON
#import pandas as pd
#import re

def count_words(text):
    words = re.findall(r'\w+', text)
    return len(words)

def main():
    # Carregar o arquivo CSV
    df = pd.read_csv('imdb-reviews-pt-br.csv', encoding='UTF-8', quotechar='"', escapechar='\\')

    # Filtrar os textos negativos
    negative_reviews = df[df['sentiment'] == 'neg']

    # Contar palavras nos textos negativos em português
    negative_reviews['word_count_pt'] = negative_reviews['text_pt'].apply(count_words)

    # Contar palavras nos textos negativos em inglês
    negative_reviews['word_count_en'] = negative_reviews['text_en'].apply(count_words)

    # Somar o total de palavras em português e inglês
    total_words_pt = negative_reviews['word_count_pt'].sum()
    total_words_en = negative_reviews['word_count_en'].sum()

    # Calcular a diferença
    difference = total_words_pt - total_words_en

    # Exibir os resultados
    print(f"Total de palavras nos textos negativos em português: {total_words_pt}")
    print(f"Total de palavras nos textos negativos em inglês: {total_words_en}")
    print(f"Diferença de palavras (português - inglês): {difference}")

if __name__ == "__main__":
    main()


<ipython-input-4-45a13645567f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews['word_count_pt'] = negative_reviews['text_pt'].apply(count_words)


Total de palavras nos textos negativos em português: 5458578
Total de palavras nos textos negativos em inglês: 5493543
Diferença de palavras (português - inglês): -34965


<ipython-input-4-45a13645567f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews['word_count_en'] = negative_reviews['text_en'].apply(count_words)
